# DataLoader
<p> Tutorial link: https://github.com/RaivoKoot/Video-Dataset-Loading-Pytorch or https://video-dataset-loading-pytorch.readthedocs.io/en/latest/ </p>

## Database preparation
<li> Input : 3 folders named ( tiktok, youtube, vimizer) which contains the mp4 videos  </li>
<li> For each Folders : Create an homologue folder which contains num_videos folders. All of these folders named by the index of the video contain each frames of the same video. </li>
<li> Annotations.txt file generation </li>

### Creation of the architecture

In [46]:
### Useful Path ###
dataset_video= 'dataset_video'
dataset = 'dataset'

In [47]:
##### Librairies #####
import os
import cv2
import subprocess
import shutil

In [48]:
### Nettoyer la base de donnée ######
# remove all the folder of the database
shutil.rmtree(dataset)


In [49]:
####### Database creation ########
os.makedirs(dataset)
classe=-1

for folder in os.listdir(dataset_video):

    classe=classe+1

    folder_path = dataset + '/'+ folder
    os.makedirs(folder_path)
    print('creation of the folder ' + folder)
    
    folder_video_path = dataset_video + '/'+ folder

    for i, video in  enumerate(os.listdir(folder_video_path)) :

        video_path = folder_video_path + '/' + video
        output_folder_video_path = folder_path + '/' + str(i)

        # Creation of the frame folder for the video i 
        os.makedirs(output_folder_video_path)

        # Extraction of the frames in this folder
        command = "ffmpeg -i " + video_path + " -vf fps=2 " + output_folder_video_path +"/img_%05d.png"
        subprocess.run(command, shell=True)

        #Ajout de la ligne dans annotation.txt
        elements = os.listdir(output_folder_video_path)
        compteur= len(elements)
       

        with open('annotation.txt', 'a') as file:
            file.write(folder + '/' + str(i)+ " " + str(compteur) + " " + str(classe) + "\n")

        

creation of the folder vimizer


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.0.40.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enabl

creation of the folder youtube


[swscaler @ 0x1402e8000] [swscaler @ 0x148008000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1402e8000] [swscaler @ 0x150008000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1402e8000] [swscaler @ 0x150018000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1402e8000] [swscaler @ 0x150028000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1402e8000] [swscaler @ 0x150038000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1402e8000] [swscaler @ 0x150048000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1402e8000] [swscaler @ 0x150058000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1402e8000] [swscaler @ 0x150068000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1402e8000] [swscaler @ 0x150078000] No accelerated colorsp

creation of the folder tiktok


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.0.40.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enabl